<a href="https://colab.research.google.com/github/lavanaythakral/IAI-Pipeline-demo/blob/main/Demo1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Prerequisites

In [ ]:
!pip install transformers

In [ ]:
!pip install yake

In [ ]:
!pip install -U spacy
!python -m spacy download en_core_web_sm

In [ ]:
import csv
import requests
import yake
import pandas as pd
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk import wordpunct_tokenize, WordNetLemmatizer, sent_tokenize, pos_tag
from nltk.corpus import stopwords as sw, wordnet as wn
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
import string 
import json
import ast
import torch
import spacy
nlp = spacy.load("en_core_web_sm")

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Data

In [14]:
df = pd.read_csv('/content/drive/My Drive/Thesis project/Thesis project/datasets/WOF_split_into_sentences.csv')
sentences = list(df['Sentences'])
len(sentences)

3085

#Test Questions

In [15]:
test_questions = ["What is your name?","What is SLV?", "When is your birthday?" , "Who is Narayan?", "Who is your sister?", "where are you from?", "Where is Pentagon?" , "where did Trishul take off from?", "What is Prithvi?" ,"What is Astra?", "Did you have childhood friends?","Which school did you go to as a child?" , "when did you start liking Science?", "What was your role in Nike-Apache?" , "What are sounding rockets used for?" , "Are you religious?" , "What is SAM?" , "Who is Indira Gandhi?" , "Who conferred the Honorary Doctor of Science degree on to you?" , "What is TCV?"]
test_questions_B = ["How can I overcome difficulty?","Why did I fail?","How can I become successful?","How can I be more creative?","What is love?","Do you need to fail to succeed?","Is there an afterlife?","What is the meaning of life?","How do you judge a person?","What is the most important thing in you life?","What is your definition of success?","What type of person do I want to be?","How can you love yourself more today?","How can you change someone's life for the better today?","What inspire you the most?","What should I do next?","What should I have for dinner?","cheer up","what is your mission?"]
test_questions_C = ["Why do we create Art?", "What is free will?", "What is freedom","What is truth?","What is the soul?","What do you fear most?","Is there such a thing as luck","Are clowns funny or scary?","What are rights?","What is mercy?","What is self-esteem?","what is age?","Can money buy happiness?","Do you like music?","What are dreams?","How to be happy?","What is beauty?","Should you forgive people?","Is our universe real?","Does god exist?","What is friendship?","What is patriotism?","Are wars necessary?","What is time?","What do you think about time travel?","What is philosophy?","Who are you?","Do animals have feelings?","What is the goal of life?","Is lying okay?","Is humanity headed in the right direction?","What is wisdom?","What is waste of human potential?","What is funny?","Are animals moral?","is privacy a right?","What is your definition of evil?","Is censorship necessary?","Who are you?","How to be confident?","Is everyone's life value equal?"]

#Fetching Answers from memory

##Google API

In [33]:
def keys(query):
  kw_extractor = yake.KeywordExtractor()
  keywords = kw_extractor.extract_keywords(query)
  txt = "A P J Abdul Kalam"
  for kw in keywords:
    # print(kw[1])
    txt += " "
    txt += kw[1]
  return txt,keywords

def get_ans(question):
  API = '4e1eb040-18e7-11eb-a969-4d03007837e9'
  headers = { 'apikey': API }
  params = (
    ("q",question),
    ("device","desktop"),
    ("gl","IN"),
    ("hl","en"),
    ("location","Navi Mumbai,Maharashtra,India"),
    ("num","50"),
  )

  response = requests.get('https://app.zenserp.com/api/v2/search', headers=headers, params=params)
  data = response.json()
  if 'answer_box' in data.keys():
    res = data['answer_box']['answer']
  else:
    res = "No clue"
  return res

def master(query):
  txt,keywords = keys(query)
  # print(txt)
  res = get_ans(txt)
  return res

##Context Extraction

In [27]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(sentences)
X = normalize(X)

In [28]:
def get_context(query,X,data_sentences):
  Question = vectorizer.transform([query])
  Question = normalize(Question)
  cosineSimilarities = cosine_similarity(Question, X).flatten()
  idx = cosineSimilarities.argsort()[::-1][:20]
  temp = ""
  # print(query)
  for i in idx:
    if(cosineSimilarities[i] != 0):
      # if(i-1 >=0):
      #   temp = temp + data_sentences[i-1]
      temp = temp + data_sentences[i]
      # if(i+1 < len(data_sentences)):
      #   temp = temp + data_sentences[i+1]
  return temp

##BERT for Q/A

In [12]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
tokenizer = AutoTokenizer.from_pretrained("deepset/bert-large-uncased-whole-word-masking-squad2")
model = AutoModelForQuestionAnswering.from_pretrained("deepset/bert-large-uncased-whole-word-masking-squad2")

In [29]:
def qna(questions,X,data_sentences):
  answers = []
  for question in questions:
    text = get_context(question,X,data_sentences)
    inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]
    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    answer_start_scores, answer_end_scores = model(**inputs)
    answer_start = torch.argmax(answer_start_scores)  # Get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
    print(f"Question: {question}")
    print(f"Answer: {answer}")
    answers.append(answer)
  return answers

def Fetching_answers(test_questions,X,sentences):
  answers = qna(test_questions,X,sentences)
  for idx,ans in enumerate(answers):
    if ans == '[CLS]' or ans == '':
      res = master(test_questions[idx])
      if res == 'No clue':
        answers[idx] = 'PASS' 
      else:
        answers[idx] = res
  return answers

##NER

In [29]:
def ner(sentence):
  doc = nlp(sentence)
  entities = []
  for ent in doc.ents:
    entities.append([ent.text,ent.label_])
  return entities

In [30]:
def entities(sentences,answers):
  for idx,sen in enumerate(sentences):
    if(answers[idx] == 'PASS' and len(ner(sen)) == 0):
      print(sen)
      answers[idx] = 'GPT2'
  return answers

##Final extraction

In [35]:
def phase_one_end(test_questions,X,sentences):
  ans = Fetching_answers(test_questions,X,sentences)
  final = entities(test_questions,ans)
  return final

In [ ]:
answers = phase_one_end(test_questions,X,sentences)

In [ ]:
with open('/content/drive/My Drive/Thesis project/Thesis project/datasets/Questions_for_GPT2/T1.csv',"w") as f:
  writer= csv.writer(f)
  writer.writerow(["Question",'Answer'])
  for idx,ans in enumerate(answers):
    writer.writerow([test_questions[idx],answers[idx]])

#GPT2

##Prereq

In [16]:
!cp -r "/content/drive/My Drive/Thesis project/Thesis project/Pytorch_out/Pytorch_out" "/content"

In [17]:
from transformers import GPT2Model,GPT2Tokenizer

In [18]:
tokenizer = GPT2Tokenizer.from_pretrained('/content/Pytorch_out/')
model = GPT2Model.from_pretrained('/content/Pytorch_out/')

In [68]:
!git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 49014, done.
remote: Total 49014 (delta 0), reused 0 (delta 0), pack-reused 49014
Receiving objects: 100% (49014/49014), 36.22 MiB | 29.98 MiB/s, done.
Resolving deltas: 100% (34215/34215), done.


In [20]:
cd transformers/examples/text-generation

/content/question-generation/transformers/examples/text-generation


##Generating Function

In [21]:
def gener():
  generated = !python run_generation.py \
  --model_type gpt2 \
  --model_name_or_path "/content/Pytorch_out/" \
  --length 300 \
  --prompt "When did you start liking science?" \
  --stop_token "." \
  --k 40 \
  --num_return_sequences 5 \
  --seed 556
  return generated

def clean(input_st, sub):
  return input_st.replace(sub, '').lstrip()

In [ ]:
# 5,7,9,11,13 are the answers
results = []

##BERT For NSP

In [22]:
from torch.nn.functional import softmax
from transformers import BertForNextSentencePrediction, BertTokenizer
model = BertForNextSentencePrediction.from_pretrained('bert-base-cased')
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [62]:
def top_result(seq_A,seq_B):
  # print(seq_B)
  response = seq_B[0]
  max_prob = -1
  for seq in seq_B:
    encoded = tokenizer.encode_plus(seq_A, text_pair=seq, return_tensors='pt')
    seq_relationship_logits = model(**encoded)[0]
    probs = softmax(seq_relationship_logits, dim=1)
    if probs[0][0] > max_prob:
      max_prob = probs[0][0]
      response = seq
  if max_prob >= 0.97:
    return response
  else:
    return -1

##Final improv

In [35]:
words = {'science' : 'positive'}

In [61]:
def master_GPT2(inp):
  generate = gener()
  # print(generate)
  generation_cleaned = []
  # print(generate)
  for gen in generate[5:14:2]:
    generation_cleaned.append(re.sub(r"^\W+", "",clean(clean(gen,inp),'<|endoftext|>')))
  
  # print(generation_cleaned)
  candidates = []
  for gen in generation_cleaned:
    if len(ner(gen)) == 0:
      # print(gen)
      candidates.append(gen)
  print(candidates)
  res = top_result(inp,candidates)
  polarity = sid.polarity_scores(res)['compound']
  # print(res)
  # print(polarity)
  return res,polarity

In [42]:
def regeneration(inp,words):
  flg = 1
  resp = ""
  pol = ""
  while(flg == 1):
    flg = 0
    resp,pol = master_GPT2(inp)
    txt,keywords = keys(resp)
    print(resp,pol)
    if pol > 0.0:
      polarity = 'positive'
    elif pol < 0.0:
      polarity = 'negative'
    else:
      polarity = 'neutral'
    for x in keywords:
      if x in words.keys() and polarity != words[x]:
        flg = 1
  return resp


In [63]:
cd ~

/root


In [65]:
cd /content

/content


In [70]:
cd transformers/examples/text-generation

/content/transformers/examples/text-generation


In [71]:
GPT2_generation = regeneration("When did you start liking science?",words)

['When did you start liking Science?I would say, somewhere in the middle of']
When did you start liking Science?I would say, somewhere in the middle of 0.4019


#Question-Generation

##Prereq

In [46]:
cd ~

/root


In [47]:
cd /content

/content


In [2]:
!git clone https://github.com/sumehta/question-generation.git

Cloning into 'question-generation'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 112 (delta 4), reused 0 (delta 0), pack-reused 103
Receiving objects: 100% (112/112), 51.13 MiB | 50.88 MiB/s, done.
Resolving deltas: 100% (62/62), done.


In [48]:
cd question-generation/

/content/question-generation


In [ ]:
!unzip QuestionGeneration.zip

##Generation

In [74]:
cd ~

/root


In [75]:
cd /content/question-generation

/content/question-generation


In [76]:
def generate_q():
  q = !python question.py -s 'I would say, somewhere in the middle of'
  l = ast.literal_eval(q[0])
  if(len(l) == 0):
    return "No Question"
  return l[0]['Q']

In [77]:
q = generate_q()
print(q)
# third_col.append(q)

Who would say?
